In [1]:
import pandas as pd
import unidecode

In [2]:
from formula_one import FormulaOne

In [3]:
f1 = FormulaOne()
f1

Data
    Races
        (144, 7)
    Circuits
        (27, 7)
    Results
        (2947, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

In [4]:
f1.update()

'Data is up to date. | Most Recent RaceID: `202106`'

In [5]:
fantasy_rosters = pd.read_csv('./draft/fantasy_rosters.csv', index_col=0)
fantasy_rosters

,OverallPick,TeamPick,Team,DriverID,LastName,FirstName,Car,Nationality
0,1,1,Carlaniel & the Romeos,norris,Norris,Lando,MCLAREN MERCEDES,GBR
1,2,1,Scuderia Spaghetti,hamilton,Hamilton,Lewis,MERCEDES,GBR
2,3,1,3 Orange Whips,max_verstappen,Verstappen,Max,RED BULL RACING HONDA,NED
3,4,1,Deep Fried,bottas,Bottas,Valtteri,MERCEDES,FIN
4,5,2,Deep Fried,perez,Perez,Sergio,RED BULL RACING HONDA,MEX
5,6,2,3 Orange Whips,gasly,Gasly,Pierre,ALPHATAURI HONDA,FRA
6,7,2,Scuderia Spaghetti,leclerc,Leclerc,Charles,FERRARI,MON
7,8,2,Carlaniel & the Romeos,sainz,Sainz,Carlos,FERRARI,ESP
8,9,3,Carlaniel & the Romeos,ricciardo,Ricciardo,Daniel,MCLAREN MERCEDES,AUS
9,10,3,Scuderia Spaghetti,russell,Russell,George,WILLIAMS MERCEDES,GBR
